In [1]:
import pandas as pd

In [2]:
import numpy as np

In [32]:
fore_sj = pd.read_csv('forecast_sj_three_feat_1.csv')

In [33]:
fore_iq = pd.read_csv('forecast_iq_three_feat_1.csv')

In [34]:
fore_sj.shape

(260, 29)

In [35]:
fore_sj.head()

,Unnamed: 0,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,avg_ndvi,avg_ndvi_lower,avg_ndvi_upper,...,reanalysis_relative_humidity_percent,reanalysis_relative_humidity_percent_lower,reanalysis_relative_humidity_percent_upper,yearly,yearly_lower,yearly_upper,additive_terms,additive_terms_lower,additive_terms_upper,yhat
0,0,2008-04-29,33.312192,-19.906697,31.618175,33.312192,33.312192,-0.025798,-0.025798,-0.025798,...,0.004070,0.004070,0.004070,-0.705459,-0.705459,-0.705459,0.0,0.0,0.0,6.102823
1,1,2008-05-06,33.412208,-21.571100,32.883723,33.412208,33.412208,-0.028472,-0.028472,-0.028472,...,-0.006561,-0.006561,-0.006561,-0.716601,-0.716601,-0.716601,0.0,0.0,0.0,5.170162
2,2,2008-05-13,33.512223,-19.317271,32.941178,33.512223,33.512223,-0.019766,-0.019766,-0.019766,...,-0.005790,-0.005790,-0.005790,-0.741566,-0.741566,-0.741566,0.0,0.0,0.0,8.276209
3,3,2008-05-20,33.612238,-23.619476,30.113521,33.612238,33.612238,-0.019771,-0.019771,-0.019771,...,-0.107080,-0.107080,-0.107080,-0.801814,-0.801814,-0.801814,0.0,0.0,0.0,3.825585
4,4,2008-05-27,33.712254,-24.757820,30.071061,33.712254,33.712254,-0.019259,-0.019259,-0.019259,...,-0.086481,-0.086481,-0.086481,-0.861482,-0.861482,-0.861482,0.0,0.0,0.0,2.623305


In [36]:
submi = pd.read_csv('submission_format.csv')

In [37]:
submi_sj = submi[submi['city'] == 'sj']
submi_iq = submi[submi['city'] == 'iq']


In [38]:
submission_sj = pd.concat([submi_sj, pd.DataFrame(fore_sj['yhat'])], axis=1)

In [39]:
submission_sj.head()

,city,year,weekofyear,total_cases,yhat
0,sj,2008,18,0,6.102823
1,sj,2008,19,0,5.170162
2,sj,2008,20,0,8.276209
3,sj,2008,21,0,3.825585
4,sj,2008,22,0,2.623305


In [40]:
submission_sj = submission_sj.drop(['total_cases'],axis = 1)

In [41]:
submission_sj.rename(columns = {'yhat':'total_cases'}, inplace = True)

In [42]:
submi_iq.reset_index(inplace=True)

In [43]:
submission_iq = pd.concat([submi_iq, pd.DataFrame(fore_iq['yhat'])], axis=1)

In [44]:
submission_iq.shape

(156, 6)

In [45]:
submission_iq = submission_iq.drop(['index','total_cases'],axis = 1)

In [46]:
submission_iq.head()

,city,year,weekofyear,yhat
0,iq,2010,26,3.838950
1,iq,2010,27,3.445873
2,iq,2010,28,5.167285
3,iq,2010,29,4.290463
4,iq,2010,30,2.899982


In [47]:
submission_iq.rename(columns = {'yhat':'total_cases'}, inplace = True)

In [48]:
main_submission = submission_sj.append(submission_iq)

/var/folders/dl/zfkvqms549vchyzbsj30cgbw0000gn/T/ipykernel_47960/2663588300.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  main_submission = submission_sj.append(submission_iq)


In [49]:
main_submission.shape

(416, 4)

In [50]:
main_submission.set_index('city',inplace=True)

In [51]:
main_submission_round = main_submission.copy()

In [52]:
main_submission_round['total_cases'] = main_submission_round['total_cases'].round(0)

In [53]:
main_submission_round.head()

,year,weekofyear,total_cases
city,,,
sj,2008,18,6.0
sj,2008,19,5.0
sj,2008,20,8.0
sj,2008,21,4.0
sj,2008,22,3.0


In [54]:
main_submission_round['total_cases'] = main_submission_round['total_cases'].astype('int64')

In [55]:
main_submission_round[main_submission_round['total_cases'] < 0]

,year,weekofyear,total_cases
city,,,
sj,2009,14,-2
sj,2011,11,-3
sj,2013,12,-1
iq,2013,25,-1


In [56]:
main_submission_round['total_cases'] = main_submission_round['total_cases'].apply(lambda x: x if x>=0 else 0)

In [57]:
main_submission_round[main_submission_round['total_cases'] < 0]

,year,weekofyear,total_cases
city,,,


In [58]:
main_submission_round.to_csv('submission_prophet_new_three_feat_1.csv')